In [1]:
import pandas as pd
import numpy as np
print('ok')

ok


In [15]:
#读入
spacialMatrix='空间矩阵.xlsx'
flowData='rd.xlsx'
spacialflow=pd.read_excel(r'{}'.format(spacialMatrix))
flow = pd.read_excel(r'{}'.format(flowData))
flow=flow.fillna(value=0)
print('ok')

ok


In [16]:
W_OR=pd.DataFrame(index=(i for i in range(1,flow.shape[0]+1)),columns=([i for i in range(1,flow.shape[0]+1)]))
print(flow.head())

   ID  转出地        转入地  流量
0   1  常州市        安庆市   1
1   2  常州市        安庆市   1
2   3  常州市        安庆市   2
3   4  常州市        鞍山市   2
4   5  常州市  巴音郭楞蒙古自治州   2


In [17]:
#建立空间索引字典
def CreatSpatialDict(spacialflow):
    citys=spacialflow.columns
    global sp_dict
    sp_dict={}
    for row in spacialflow.itertuples():
        stocks_list=list(row)
        ad_index=[x-1 for x in range(len(stocks_list)) if stocks_list[x]==1]
        sp_dict[row[0]]=list(citys[ad_index])
    print(sp_dict)
    
    
CreatSpatialDict(spacialflow)

{'淮安市': ['扬州市', '宿迁市', '连云港市', '盐城市', '滁州市'], '南京市': ['扬州市', '镇江市', '常州市', '合肥市', '滁州市', '马鞍山市', '宣城市'], '扬州市': ['淮安市', '南京市', '镇江市', '泰州市', '盐城市', '滁州市'], '镇江市': ['南京市', '扬州市', '常州市', '泰州市'], '常州市': ['南京市', '镇江市', '泰州市', '无锡市', '宣城市'], '泰州市': ['扬州市', '镇江市', '常州市', '无锡市', '南通市', '盐城市'], '无锡市': ['常州市', '泰州市', '苏州市', '宣城市', '湖州市'], '苏州市': ['无锡市', '湖州市', '嘉兴市', '上海市'], '南通市': ['泰州市', '盐城市'], '徐州市': ['宿迁市', '连云港市', '菏泽市', '济宁市', '枣庄市', '宿州市', '临沂市'], '宿迁市': ['淮安市', '徐州市', '连云港市', '蚌埠市', '滁州市', '宿州市'], '连云港市': ['淮安市', '徐州市', '宿迁市', '盐城市', '临沂市', '日照市'], '盐城市': ['淮安市', '扬州市', '泰州市', '南通市', '连云港市'], '阿里地区': ['和田地区'], '保山市': ['大理白族自治州'], '大理白族自治州': ['保山市', '楚雄彝族自治州'], '楚雄彝族自治州': ['大理白族自治州', '攀枝花市', '玉溪市', '凉山彝族自治州', '昆明市'], '攀枝花市': ['楚雄彝族自治州', '凉山彝族自治州'], '玉溪市': ['楚雄彝族自治州', '昆明市', '红河哈尼族彝族自治州'], '凉山彝族自治州': ['楚雄彝族自治州', '攀枝花市', '雅安市', '昆明市', '曲靖市', '乐山市', '宜宾市'], '雅安市': ['凉山彝族自治州', '阿坝藏族羌族自治州', '乐山市', '眉山市', '成都市'], '阿坝藏族羌族自治州': ['雅安市', '成都市', '德阳市', '绵阳市', '陇南市'], '昆明市': ['楚雄彝族自治州', '玉溪市', '凉山彝

In [18]:
def getAdjacentFlow(origin, destination):
    Adlist=[]
    sameOri=flow[flow['转出地'].str.contains(origin)]
    sameDes=flow[flow['转入地'].str.contains(destination)]
    Adlist1=sameOri[sameOri['转入地'].isin(sp_dict[destination])].ID.tolist()
    Adlist2=sameDes[sameDes['转出地'].isin(sp_dict[origin])].ID.tolist()
    Adlist=Adlist1+Adlist2
    return Adlist

# getAdjacentFlow('常州市','阿拉善盟')

for row in flow.itertuples():
    try:
        netAdjacentList=getAdjacentFlow(row.转出地, row.转入地)
        print(row.ID, netAdjacentList)
        W_OR.loc[row.ID,netAdjacentList]=1
    except Exception as ex:
        print('ID={}出错'.format(row.ID))
        print('发生错误{}'.format(ex))

print('正在保存')    
W_OR.dropna(axis=0,how='all').dropna(axis=1,how='all').fillna(value=0).to_excel('RD-网络邻接.xlsx')
print('保存成功')

1 [38, 39, 45]
2 [38, 39, 45]
3 [38, 39, 45]
4 [23, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 957]
5 [40, 139, 140, 166]
6 [40, 139, 140, 166]
7 [22, 43, 44, 122, 123, 124, 125, 126, 127, 256]
8 [26, 894, 1286]
9 [133, 134, 135, 136, 169, 258, 259, 260, 261, 262, 263, 264, 265, 266, 848, 849, 850, 897, 898, 899, 900, 901, 902, 903, 904, 1287, 1288, 1289, 1290, 1291]
10 [133, 134, 135, 136, 169, 258, 259, 260, 261, 262, 263, 264, 265, 266, 848, 849, 850, 897, 898, 899, 900, 901, 902, 903, 904, 1287, 1288, 1289, 1290, 1291]
11 [133, 134, 135, 136, 169, 258, 259, 260, 261, 262, 263, 264, 265, 266, 848, 849, 850, 897, 898, 899, 900, 901, 902, 903, 904, 1287, 1288, 1289, 1290, 1291]
12 [133, 134, 135, 136, 169, 258, 259, 260, 261, 262, 263, 264, 265, 266, 848, 849, 850, 897, 898, 899, 900, 901, 902, 903, 904, 1287, 1288, 1289, 1290, 1291]
13 [133, 134, 135, 136, 169, 258, 259, 260, 261, 262, 263, 264, 265, 266, 848, 849, 850, 897, 898, 899, 900, 901, 902, 903, 904, 1287, 1288, 1289,

98 [48, 49, 50, 116, 117, 118, 119, 120, 121, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 868, 869, 870, 871, 872, 873, 874, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1322, 1323, 1324, 1325, 1326, 1327, 1328]
99 [48, 49, 50, 116, 117, 118, 119, 120, 121, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 868, 869, 870, 871, 872, 873, 874, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1322, 1323, 1324, 1325, 1326, 1327, 1328]
100 [48, 49, 50, 116, 117, 118, 119, 120, 121, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 868, 869, 870, 871, 872, 873, 874, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1322, 1323, 1324, 1325, 1326, 1327, 1328]
101 [48, 49, 50, 116, 117, 118, 119, 120, 121, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 868, 869, 870, 871, 872, 873, 874, 985, 986, 987, 988, 989, 990, 991, 992, 993

163 [42, 55, 56, 57, 58, 59, 60, 61, 81, 82, 83, 84, 85, 86, 130, 131, 132, 165, 432, 433, 434, 1049, 1050, 1051, 1052]
164 [42, 55, 56, 57, 58, 59, 60, 61, 81, 82, 83, 84, 85, 86, 130, 131, 132, 165, 432, 433, 434, 1049, 1050, 1051, 1052]
165 [22, 42, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 130, 131, 132, 158, 159, 160, 161, 162, 163, 164, 177, 178, 179, 180, 435, 436, 437, 438, 439, 440, 441, 885, 886, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1343, 1344]
166 [5, 6]
167 [114, 183, 888, 1061, 1062, 1063]
168 []
169 [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 1066]
170 [64, 91, 92, 442]
171 [32]
172 [154, 187, 1067, 1068, 1069]
173 [154, 187, 1067, 1068, 1069]
174 [154, 187, 1067, 1068, 1069]
175 [154, 187, 1067, 1068, 1069]
176 [63]
177 [67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 130, 131, 132, 165, 443, 444, 445, 1070, 1071, 1072, 1073]
178 [67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 130, 131, 132, 165, 443, 444, 445, 1070, 1071, 1072, 1073]
179 [67, 68, 69,

289 [26]
290 [26]
291 [26]
292 [26]
293 [294, 295, 296, 297, 298, 299, 448, 1221]
294 [286, 287, 288, 293, 318]
295 [286, 287, 288, 293, 318]
296 [286, 287, 288, 293, 318]
297 [286, 287, 288, 293, 318]
298 [286, 287, 288, 293, 318]
299 [286, 287, 288, 293, 318]
300 [326, 327]
301 [365, 34]
302 [365, 34]
303 [365, 34]
304 [365, 34]
305 [365, 34]
306 [365, 34]
307 [380, 36, 37]
308 [380, 36, 37]
309 [277, 278, 279, 317, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 414, 38, 39, 1225, 1226]
310 [277, 278, 279, 317, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 414, 38, 39, 1225, 1226]
311 [277, 278, 279, 317, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 414, 38, 39, 1225, 1226]
312 [277, 278, 279, 330, 331, 332, 333, 394, 395, 396, 397, 398, 399, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 42, 1228, 1229, 1298, 1299, 1300, 1301]
313 [277, 278, 279, 330, 331, 332, 333, 394, 395, 396, 397, 398, 399, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 42, 1228, 1229, 1298, 1

409 [270, 271, 272, 354, 419, 420, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 443, 444, 445, 130, 131, 132, 1267, 1268, 1269, 1270, 1336]
410 [258, 259, 260, 261, 262, 263, 264, 265, 266, 133, 134, 135, 136]
411 [258, 259, 260, 261, 262, 263, 264, 265, 266, 133, 134, 135, 136]
412 [258, 259, 260, 261, 262, 263, 264, 265, 266, 133, 134, 135, 136]
413 [258, 259, 260, 261, 262, 263, 264, 265, 266, 133, 134, 135, 136]
414 [276, 309, 310, 311]
415 [424, 425]
416 [269, 139, 140]
417 [269, 139, 140]
418 [269, 139, 140]
419 [270, 271, 272, 388, 389, 390, 391, 392, 393, 404, 405, 406, 407, 408, 409, 141, 142, 143, 144, 145, 146, 147, 1337, 1338, 1339, 1340, 1341]
420 [270, 271, 272, 388, 389, 390, 391, 392, 393, 404, 405, 406, 407, 408, 409, 141, 142, 143, 144, 145, 146, 147, 1337, 1338, 1339, 1340, 1341]
421 [149, 150, 1273]
422 [149, 150, 1273]
423 [149, 150, 1273]
424 [257, 415, 151, 152, 1274, 1342]
425 [257, 415, 151, 152, 1274, 1342]
426 [154]
427 [284, 155]
428 [320, 330, 331, 332

...

873 [876, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1176, 1177, 1178, 1179, 1180, 1181, 1182, 1183, 1184, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1322, 1323, 1324, 1325, 1326, 1327, 1328]
874 [876, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1176, 1177, 1178, 1179, 1180, 1181, 1182, 1183, 1184, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1322, 1323, 1324, 1325, 1326, 1327, 1328]
875 [1006, 1186]
876 [868, 869, 870, 871, 872, 873, 874, 878, 879, 880, 881, 882, 883, 884, 116, 117, 118, 119, 120, 121, 511, 512, 513, 514, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1187, 1188, 1189, 1190, 1265, 1266, 

975 [981]
976 [87, 697]
977 [896, 965]
978 [953, 90]
979 [953, 90]
980 [938, 939, 966]
981 [975, 984, 91, 702]
982 [951, 952, 703]
983 [1030]
984 [966, 981, 867]
985 [1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 868, 869, 870, 871, 872, 873, 874]
986 [1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 868, 869, 870, 871, 872, 873, 874]
987 [1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 868, 869, 870, 871, 872, 873, 874]
988 [10

1054 [919, 943, 960, 961, 962, 963, 964, 1025, 1026, 1027, 1049, 1050, 1051, 1052, 1070, 1071, 1072, 1073, 165, 804, 805, 806, 885, 886]
1055 [919, 943, 960, 961, 962, 963, 964, 1025, 1026, 1027, 1049, 1050, 1051, 1052, 1070, 1071, 1072, 1073, 165, 804, 805, 806, 885, 886]
1056 [919, 943, 960, 961, 962, 963, 964, 1025, 1026, 1027, 1049, 1050, 1051, 1052, 1070, 1071, 1072, 1073, 165, 804, 805, 806, 885, 886]
1057 [919, 943, 960, 961, 962, 963, 964, 1025, 1026, 1027, 1049, 1050, 1051, 1052, 1070, 1071, 1072, 1073, 165, 804, 805, 806, 885, 886]
1058 [919, 943, 960, 961, 962, 963, 964, 1025, 1026, 1027, 1049, 1050, 1051, 1052, 1070, 1071, 1072, 1073, 165, 804, 805, 806, 885, 886]
1059 [919, 943, 960, 961, 962, 963, 964, 1025, 1026, 1027, 1049, 1050, 1051, 1052, 1070, 1071, 1072, 1073, 165, 804, 805, 806, 885, 886]
1060 [1038, 887]
1061 [890, 891, 1081, 167, 808, 888]
1062 [890, 891, 1081, 167, 808, 888]
1063 [890, 891, 1081, 167, 808, 888]
1064 [920, 921]
1065 [920, 921]
1066 [895, 897, 89

1186 [875]
1187 [1150, 1151, 1176, 1177, 1178, 1179, 1180, 1181, 1182, 1183, 1184, 1198, 217, 249, 511, 512, 513, 514, 876, 1265, 1266]
1188 [1150, 1151, 1176, 1177, 1178, 1179, 1180, 1181, 1182, 1183, 1184, 1198, 217, 249, 511, 512, 513, 514, 876, 1265, 1266]
1189 [1150, 1151, 1176, 1177, 1178, 1179, 1180, 1181, 1182, 1183, 1184, 1198, 217, 249, 511, 512, 513, 514, 876, 1265, 1266]
1190 [1150, 1151, 1176, 1177, 1178, 1179, 1180, 1181, 1182, 1183, 1184, 1198, 217, 249, 511, 512, 513, 514, 876, 1265, 1266]
1191 [1144, 1156, 1157, 1158, 515, 877]
1192 []
1193 [1185]
1194 [1140, 1172, 1198, 516, 517, 518, 1267, 1268, 1269, 1270]
1195 [1140, 1172, 1198, 516, 517, 518, 1267, 1268, 1269, 1270]
1196 [1174, 1272]
1197 []
1198 [1140, 1150, 1187, 1188, 1189, 1190, 1194, 1195, 250, 251, 522, 523, 524, 878, 879, 880, 881, 882, 883, 884]
1199 [525, 1274]
1200 [525, 1274]
1201 [525, 1274]
1202 [1134, 532, 888]
1203 [1134, 532, 888]
1204 [1134]
1205 [1134]
1206 [1217, 1225, 1226, 1230, 1235]
1207 [12

1292 [1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1336, 1337, 1338, 1339, 1340, 1341, 270, 271, 272, 1210, 1211]
1293 [1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1336, 1337, 1338, 1339, 1340, 1341, 270, 271, 272, 1210, 1211]
1294 [20, 273, 274, 275, 1212, 1213, 1214, 1215, 1216]
1295 [1298, 1299, 1300, 1301, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1343, 1344, 22, 277, 278, 279, 1220]
1296 [1329, 1330, 1331, 1332, 286, 287, 288]
1297 [31, 1224]
1298 [1295, 1343, 1344, 42, 312, 313, 314, 315, 316, 1228, 1229]
1299 [1295, 1343, 1344, 42, 312, 313, 314, 315, 316, 1228, 1229]
1300 [1295, 1343, 1344, 42, 312, 313, 314, 315, 316, 1228, 1229]
1301 [1295, 1343, 1344, 42, 312, 313, 314, 315, 316, 1228, 1229]
1302 [46, 47, 320]
1303 [46, 47, 320]
1304 [46, 47, 320]
1305 [1235]
ID=1306出错
发生错误'陵水黎族自治县'
1307 [1292, 1293, 1295, 1343, 1344, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 858,

In [31]:
W_OR.dropna(axis=0,how='all').dropna(axis=1,how='all').fillna(value=0).to_excel('SERVT-网络邻接.xlsx')
print('保存成功')

保存成功


# 时间近邻

In [56]:
timeflowDataName='time.xlsx'
time_table = pd.read_excel(r'{}'.format(timeflowDataName))
net_table = pd.read_excel(r'ALLT-网络邻接.xlsx')
def CreatTimeDict(flow):
    global time_dict
    time_dict={}
    for row in flow.itertuples():
        time_dict[row.ID]=row.年份
    print(time_dict)
    
    
    

CreatTimeDict(time_table)

{1: 2015, 2: 2017, 3: 2001, 4: 2010, 5: 2015, 6: 2020, 7: 2015, 8: 2014, 9: 2013, 10: 2015, 11: 1998, 12: 2020, 13: 2018, 14: 2009, 15: 1993, 16: 1994, 17: 1995, 18: 1999, 19: 2002, 20: 2003, 21: 2005, 22: 2006, 23: 2007, 24: 2008, 25: 2010, 26: 2011, 27: 2014, 28: 2015, 29: 2016, 30: 2017, 31: 2018, 32: 2010, 33: 2016, 34: 2015, 35: 2002, 36: 2011, 37: 2020, 38: 2014, 39: 2006, 40: 2015, 41: 2005, 42: 2013, 43: 1999, 44: 2018, 45: 2019, 46: 2020, 47: 2015, 48: 2020, 49: 2011, 50: 2015, 51: 2010, 52: 2014, 53: 2004, 54: 2007, 55: 2016, 56: 2017, 57: 2020, 58: 2017, 59: 1996, 60: 2016, 61: 1994, 62: 2018, 63: 2012, 64: 2018, 65: 2018, 66: 2019, 67: 2007, 68: 2008, 69: 2017, 70: 2017, 71: 2001, 72: 2013, 73: 2012, 74: 2013, 75: 2016, 76: 2011, 77: 2013, 78: 2006, 79: 2007, 80: 2005, 81: 2006, 82: 2008, 83: 2015, 84: 2012, 85: 2000, 86: 2013, 87: 1984, 88: 2017, 89: 2004, 90: 2011, 91: 2014, 92: 2010, 93: 2016, 94: 2010, 95: 2016, 96: 2007, 97: 2017, 98: 2019, 99: 2001, 100: 2008, 101: 20

In [74]:

def CreatNetDict(net_table):
    global net_dict
    net_dict={}
    for index,row in net_table.iterrows():
        idxList = row.dropna().T.index.tolist()
        year=int(time_dict[index])-1
        ad_index=[x for x in idxList if time_dict[x]!=year]
        net_dict[index]=ad_index
    print('ok')
    T_OR=net_table
    for row in T_OR.itertuples():
        T_OR.loc[row[0],net_dict[row[0]]]=np.nan
#     T_OR.
             
CreatNetDict(net_table)
# print(net_dict)

ok
   1     2     3     4     5     6     7     8     9     10    ...   3394  \
1   NaN   NaN   NaN   NaN   NaN   NaN   NaN   1.0   NaN   NaN  ...    NaN   
2   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...    NaN   
3   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...    NaN   
4   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...    NaN   
5   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...    NaN   

   3395  3396  3397  3398  3399  3400  3401  3402  3403  
1   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
2   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
3   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
4   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
5   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  

[5 rows x 3403 columns]


In [50]:
for index,row in net_table.iterrows():
    idxList = row.dropna().T.index.tolist()
    print(idxList)
    ad_index=[x for x in idxList if time_dict[x]!=2014]
    print(ad_index)
    net_dict[row[0]]=ad_index
    break

[8, 58, 219, 324, 325, 332, 353, 354, 2052, 2178, 2179]
[58, 219, 324, 325, 332, 353, 354, 2052, 2178]


KeyError: 0

In [14]:
net_table.head()

,1,2,3,4,5,6,7,8,9,10,...,3394,3395,3396,3397,3398,3399,3400,3401,3402,3403
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
